In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import statsmodels.api as sm
import statsmodels.tsa.vector_ar.vecm as vecm

url = 'https://raw.githubusercontent.com/joe-ascroft/phd/master/data/df_clean.csv'

df_clean = pd.read_csv(url)
df_clean["DATE"] = pd.to_datetime(df_clean["DATE"],dayfirst=True)
for col in df_clean.columns[1:]:
    df_clean[col] = pd.to_numeric(df_clean[col],errors='coerce')

print(df_clean)

           DATE    NCF    RCF    WTI    HH  Brent  NBP_UK  GBP-USD       NBP
0    2009-01-02  74.35  81.00  46.17  5.41  46.91   54.74   1.4453  7.911572
1    2009-01-05  77.65  84.40  48.61  5.83  49.62   56.66   1.4708  8.333553
2    2009-01-06  81.25  88.50  48.56  6.10  50.53   59.88   1.4919  8.933497
3    2009-01-07  78.65  86.25  42.75  5.89  45.86   58.03   1.5111  8.768913
4    2009-01-08  79.15  84.00  41.68  5.96  44.67   55.74   1.5202  8.473595
...         ...    ...    ...    ...   ...    ...     ...      ...       ...
2851 2020-08-25  49.75  48.90  43.17  2.54  45.86   24.61   1.3148  3.235723
2852 2020-08-26  49.70  48.95  43.21  2.52  45.64   25.10   1.3193  3.311443
2853 2020-08-27  49.80  48.90  42.88  2.52  45.09   26.68   1.3206  3.523361
2854 2020-08-28  50.60  52.20  42.96  2.46  45.81   28.95   1.3344  3.863088
2855 2020-08-31  51.95  52.90  42.61  2.30  45.28   28.95   1.3379  3.873220

[2856 rows x 9 columns]


In [2]:
rank = vecm.select_coint_rank(endog=df_clean[['WTI', 'HH', 'Brent', 'NBP', 'NCF']],
                              k_ar_diff=4,
                              det_order=0)

lag_order1 = vecm.select_order(data=df_clean[['WTI', 'HH', 'Brent', 'NBP', 'NCF']],
                              maxlags=5, deterministic="ci")

In [3]:
print(lag_order1)
print(rank)

<statsmodels.tsa.vector_ar.var_model.LagOrderResults object. Selected orders are: AIC -> 5, BIC -> 2, FPE -> 5, HQIC -> 3>
Johansen cointegration test using trace test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   5          100.5          69.82
  1   5          51.03          47.85
  2   5          25.54          29.80
-------------------------------------


In [4]:
m1 = vecm.VECM(df_clean[['WTI', 'HH', 'Brent', 'NBP', 'NCF']], 
               deterministic="ci", k_ar_diff=5, coint_rank=3)
vecm_res_1 = m1.fit()
vecm_res_1.summary()

,coef,std err,z,P>|z|,[0.025,0.975]
L1.WTI,-0.3749,0.026,-14.524,0.000,-0.425,-0.324
L1.HH,-0.0561,0.210,-0.267,0.790,-0.468,0.356
L1.Brent,0.2846,0.034,8.384,0.000,0.218,0.351
L1.NBP,-0.3675,0.202,-1.821,0.069,-0.763,0.028
L1.NCF,-0.0092,0.029,-0.319,0.749,-0.066,0.047
L2.WTI,-0.2119,0.028,-7.559,0.000,-0.267,-0.157
L2.HH,-0.1024,0.210,-0.488,0.626,-0.514,0.309
L2.Brent,0.1852,0.035,5.247,0.000,0.116,0.254
L2.NBP,0.2301,0.202,1.139,0.255,-0.166,0.626
L2.NCF,0.0044,0.029,0.151,0.880,-0.052,0.061


In [10]:
granger_results = vecm_res_1.test_granger_causality(caused="HH", signif=0.05)
granger_results.summary()

Test statistic,Critical value,p-value,df
1.272,1.518,0.168,"(24, 14065)"


In [ ]:
print(vecm_res_1.summary().as_latex())